# CNN Transfer Learning on Cats-Dogs Classification

### VGG16 trained on ImageNet data is used as pre-trained model from which to extract features.  Features are then saved, and passed through neural network with ReLu hidden layer to classify cats vs. dogs.

#### Adapted from fchollet/classifier_from_little_data_script_2.py (https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069) and blog https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras import applications
import numpy as np
import tensorflow as tf

### Tensorflow config

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
print (tf.__version__)

In [ ]:
# Set random generator seed
seed = 123

# Set python built-in random generator
import random                             
random.seed(seed)

# Set numpy random generator
np.random.seed(seed)

# Set tensorflow random generator
tf.random.set_seed(seed)

### Download and extract datasets

In [ ]:
%%capture
!wget https://dse230-exercise.s3-us-west-2.amazonaws.com/data/train.gz
!tar -xzf train.gz

In [ ]:
%%capture
!wget https://dse230-exercise.s3-us-west-2.amazonaws.com/data/validation.zip
!unzip -q validation.zip

In [ ]:
%%capture
!wget https://dse230-exercise.s3-us-west-2.amazonaws.com/data/test.zip
!unzip -q test.zip

### Set dimensions, number, and location of images

In [ ]:
# Image dimensions
img_width, img_height = 150, 150

# Location of images
train_data_dir = 'train'
validation_data_dir = 'validation'
test_data_dir = 'test'

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
print (input_shape)

# Number of images
nb_train_samples = 2000
nb_validation_samples = 400
nb_test_samples = 400

# Batch size
batch_size = 16

### Method to extract features from pre-trained network and save features

In [ ]:
def save_features():
    
    # Scale pixel values in image
    datagen = ImageDataGenerator(rescale=1. / 255)

    # Load the VGG16 network's imagenet weights, not including the last fully connected layers.
    model = applications.VGG16(include_top=False, weights='imagenet')

    # Generator that will read pictures found in subfolders of training data directory,
    # and indefinitely generate batches of image data (scaled)
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,        # Generator will only yield batches of data, no labels
        shuffle=False)          # Data will be presented in order, i.e., 1000 cat images, then 1000 dog images
    
    # The predict_generator method returns the output of the model, given input provided by a generator
    # that yields batches of numpy data
    features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    
    # Save model outputs (i.e., features) from model as numpy array
    np.save('features_train.npy', features_train) 

    # Generator to generator validation input for model
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    # Get model output for validation data
    features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    
    # Save model outputs (i.e., features) for validation data
    np.save('features_validation.npy', features_validation) 
    
    generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    # Get model output for validation data
    features_test = model.predict_generator(
        generator, nb_test_samples // batch_size)
    
    # Save model outputs (i.e., features) for validation data
    np.save('features_test.npy', features_test) 
    
    
    # Print out model architecture
    model.summary()

### Call `save_features` method to extract and save features from pre-trained network

In [ ]:
<< YOUR CODE HERE >>

### Load saved features using `np.load`. Print shapes of train, validation and test data

In [ ]:
# Load saved features for train data
train_data = << YOUR CODE HERE >>
    
# Create labels for train data.  Images were generated in order, so creating labels is easy.
train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2)) 

# Load saved features for validation data
validation_data =<< YOUR CODE HERE >> 
    
# Create labels for validation data
validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

# Load saved features for test data
test_data = << YOUR CODE HERE >> 
    
# Create labels for test data
test_labels = np.array([0] * (nb_test_samples // 2) + [1] * (nb_test_samples // 2)) 
    
<< YOUR CODE HERE >>

### Create top model

In [ ]:
# Create fully connected layer on top of model
top_model = Sequential()
top_model.add(Flatten(input_shape=train_data.shape[1:]))  # Convert 3D feature maps to 1D feature vectors
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# Create model
top_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy', 
              metrics=['accuracy'])


### Print model [summary](https://www.tensorflow.org/api_docs/python/tf/keras/Model#summary)

In [ ]:
<< YOUR CODE HERE >>

### Train top model

In [ ]:
# Set up model
epochs = 50

# Train model, keeping track of history
from tensorflow.keras.callbacks import History
hist = top_model.fit(train_data, train_labels,
                 epochs=epochs,
                 batch_size=batch_size,
                 validation_data=(validation_data, validation_labels))

### Save model and weights

In [ ]:
# Save model & weights to HDF5 file
top_model_file = 'features_model' 
top_model.save(top_model_file + '.h5')

# Save model to JSON file & weights to HDF5 file
top_model_json = top_model.to_json()
with open(top_model_file + '.json','w') as json_file:
    json_file.write(top_model_json)
top_model.save_weights(top_model_file+'-wts.h5')

print (top_model.metrics_names)

### Evaluate on validation data using [`Model.evaluate`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate)

In [ ]:
<< YOUR CODE HERE >>

### Load model again and evaluate on validation data
* This is to demonstrate saving and loading model

In [ ]:
top_model2 = tf.keras.models.load_model(top_model_file+'.h5')
print (validation_labels.shape)

<< YOUR CODE HERE >>

### Evaluate on test set

In [ ]:
<< YOUR CODE HERE >>